In [1]:
# -*- coding: utf-8 -*-
"""Wait for the CSV file updates and import data to PIM."""
import pandas
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
# Reduce the amount of logs:
import logging
import os
logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s - %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

# import Akeneo API Client
try:
    import akeneo
except ModuleNotFoundError as e:
    print ("Akeneo API wrapper for python is not installed")

# ************* НАСТРОЙКИ ****************
# -------  PIM -----------
AKENEO_CLIENT_ID = '' #os.environ.get("AKENEO_CLIENT_ID")
AKENEO_SECRET = ''
AKENEO_USERNAME = ''
AKENEO_PASSWORD = ''
AKENEO_BASE_URL = ''
# ------- PIM fields mapping ----------
# TODO
# OUTER
# INNER
# UNIT
# ------- Dimensions file -------------
DIM_FILE_PATH = 'C:\ML\Dim2pim\PBM0000data.csv'
DIM_FILE_CHECK_INTERVAL = 10 # seconds
DIM_FILE_DELIMITER = ';'
DIM_TIMESTAMP_COLUMN = 21 # starts with 0 (zero)
DIM_DEPTH_COLUMN = 13
DIM_WIDTH_COLUMN = 14
DIM_HEIGHT_COLUMN = 15
DIM_WEIGHT_COLUMN = 19
DIM_BARCODE_COLUMN= 0
DIM_QTY_COLUMN = 4
# ------- Google sheets logging
GS_LOG_USERNAME = ''
GS_LOG_PASSWORD = ''
GS_LOG_BASE_URL = ''
# ************* ********* ****************

class CSVFileHandler(FileSystemEventHandler):
    
    def __init__(self, _dim_reader):
        super(FileSystemEventHandler, self).__init__()
        self.dim_reader = _dim_reader

    def on_modified(self, event):
        if not event.is_directory and not 'thumbnail' in event.src_path:
            logging.debug("Updated: " + event.src_path)
            self.dim_reader.get_new_line()

class Dim_reader():
    last_line_time = 0
    
    def __init__(self, _pim_saver):
        self.last_line_time = 0
        self.pim_saver = _pim_saver
        
    def wait_for_updates(self):
        event_handler = CSVFileHandler(self)
        observer = Observer()
        observer.schedule(event_handler, os.path.dirname(DIM_FILE_PATH), recursive=False)
        observer.start()
        logging.debug("Watchdog observer started")
        try:
            while True:
                time.sleep(DIM_FILE_CHECK_INTERVAL)
                logging.debug("Check for the updates...")
        except KeyboardInterrupt:
            observer.stop()
        observer.join()

    def get_new_line(self):
        try:
            data_src = pandas.read_csv(DIM_FILE_PATH, delimiter=DIM_FILE_DELIMITER, encoding='ISO-8859-1')
        except Exception as e:
            logging.error('Error while reading CSV file: {name}...:\n{error}\n'.format(
                name=DIM_FILE_PATH, error=e))
            raise e
        if not data_src.empty and not data_src.tail(1).empty:
            row = data_src.tail(1).values[0]
        else:
            return 
        
        if (self.last_line_time) < row[DIM_TIMESTAMP_COLUMN]:
            self.last_line_time = row[DIM_TIMESTAMP_COLUMN]
            self.pim_saver.save_to_PIM(row)
        
class PIM_saver():
    def __init__(self):
        self.last_line_time = 0
    
    def save_to_PIM(self, _new_line):
        logging.info(_new_line)
        
    @staticmethod
    def test():
        print('test')
        
if __name__ == "__main__":
    pim_saver = PIM_saver()
    dim_reader = Dim_reader(pim_saver)
    
    dim_reader.wait_for_updates()
    
    
        
    









# ---------------------------------------------------

Akeneo API wrapper for python is not installed
